In [6]:
from jupyter_dash import JupyterDash     # pip install dash ; runs Dash apps inside jupyter ntbk
from dash import Dash, html, dcc         # html things, app = dash thingz, dcc = core components like date picker
from dash.dependencies import Output, Input # used for call backs
from dash_extensions import Lottie       # pip install dash-extensions ; lottie animations
import dash_bootstrap_components as dbc  # pip install dash-bootstrap-components ; allows use of bootstrap in html
import plotly.express as px              # pip install plotly ; line charts, bar chards
import pandas as pd                      # pip install pandas ; json and csv files reading
from datetime import date                # dates
import calendar                          # use of caendar
from wordcloud import WordCloud          # pip install wordcloud
import webbrowser

# Lottie by Emil - https://github.com/thedirtyfew/dash-extensions#
url_coonections = "https://lottie.host/8c872553-bf3d-4ef4-9739-01b2718d6c09/q6LtyMNZ28.json"
url_companies = "https://assets9.lottiefiles.com/packages/lf20_EzPrWM.json"
url_msg_in = "https://assets9.lottiefiles.com/packages/lf20_8wREpI.json"
url_msg_out = "https://assets2.lottiefiles.com/packages/lf20_Cc8Bpg.json"
url_reactions = "https://assets2.lottiefiles.com/packages/lf20_nKwET0.json"
options = dict(loop=True, autoplay=True, rendererSettings=dict(preserveAspectRatio='xMidYMid slice'))

# Import App data from csv sheets **************************************
df_cnt = pd.read_csv(
    "https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Connections.csv",
    on_bad_lines='skip'
)
if "Connected On" in df_cnt.columns:
    df_cnt["Connected On"] = pd.to_datetime(df_cnt["Connected On"], errors='coerce')
    df_cnt["month"] = df_cnt["Connected On"].dt.month
    df_cnt['month'] = df_cnt['month'].apply(lambda x: calendar.month_abbr[x] if pd.notnull(x) and int(x) in range(1,13) else "")
else:
    df_cnt["Connected On"] = pd.NaT
    df_cnt["month"] = ""
    print("Warning: 'Connected On' column not found in Connections.csv")

df_invite = pd.read_csv(
    "https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Invitations.csv",
    on_bad_lines='skip'
)
if "Sent At" in df_invite.columns:
    df_invite["Sent At"] = pd.to_datetime(df_invite["Sent At"], errors='coerce')
else:
    df_invite["Sent At"] = pd.NaT
    print("Warning: 'Sent At' column not found in Invitations.csv")

df_react = pd.read_csv(
    "https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/Reactions.csv",
    on_bad_lines='skip'
)
if "Date" in df_react.columns:
    df_react["Date"] = pd.to_datetime(df_react["Date"], errors='coerce')
else:
    df_react["Date"] = pd.NaT
    print("Warning: 'Date' column not found in Reactions.csv")

df_msg = pd.read_csv(
    "https://raw.githubusercontent.com/busyizzybee/socialMediaEngagement/refs/heads/main/FOLDER/messages.csv",
    on_bad_lines='skip'
)
if "DATE" in df_msg.columns:
    df_msg["DATE"] = pd.to_datetime(df_msg["DATE"], errors='coerce')
else:
    df_msg["DATE"] = pd.NaT
    print("Warning: 'DATE' column not found in messages.csv")

if "Connected On" in df_cnt.columns:
    df_cnt["Connected On"] = pd.to_datetime(df_cnt["Connected On"], errors='coerce')
    df_cnt["month"] = df_cnt["Connected On"].dt.month
    df_cnt['month'] = df_cnt['month'].apply(lambda x: calendar.month_abbr[x] if pd.notnull(x) and int(x) in range(1,13) else "")
else:
    df_cnt["Connected On"] = pd.NaT
    df_cnt["month"] = ""
    print("Warning: 'Connected On' column not found in Connections.csv")

min_date = df_cnt["Connected On"].min().date() if df_cnt["Connected On"].notna().any() else date(2018, 1, 1)
max_date = df_cnt["Connected On"].max().date() if df_cnt["Connected On"].notna().any() else date(2025, 9, 4)

# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='assets/linkedin-logo2.png') # 150px by 45px
            ],className='mb-2'),
            dbc.Card([
                dbc.CardBody([
                    dbc.CardLink("Connectly", target="_blank",
                                 href="https://github.com/busyizzybee/socialMediaEngagement"
                    )
                ])
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.DatePickerSingle(
                        id='my-date-picker-start',
                        date=min_date,
                        className='ml-5'
                    ),
                    dcc.DatePickerSingle(
                        id='my-date-picker-end',
                        date=max_date,
                        className='mb-2 ml-2'
                    ),
                ])
            ], color="info"),
        ], width=8),
    ],className='mb-2 mt-2'),
    dbc.Row([dbc.Col([
            dbc.Card([             
                dbc.CardHeader(
                    html.Div(
                        Lottie(options=options, url=url_coonections),
                        style={"width": "180px", "height": "150px", "margin": "0 auto", "display": "block"}
                    )
                ),
                dbc.CardBody([
                    html.H6('Connections'),
                    html.H2(id='content-connections', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                   dbc.CardHeader(
                    html.Div(
                        Lottie(options=options, url=url_companies),
                        style={"width": "180px", "height": "150px", "margin": "0 auto", "display": "block"}
                    )
                ),
              #  dbc.CardHeader(Lottie(options=options, width="32%", height="32%", url=url_companies)),
                dbc.CardBody([
                    html.H6('Companies'),
                    html.H2(id='content-companies', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
                #dbc.CardHeader(Lottie(options=options, width="25%", height="25%", url=url_msg_in)),
                   dbc.CardHeader(
                    html.Div(
                        Lottie(options=options, url=url_msg_in),
                        style={"width": "180px", "height": "150px", "margin": "0 auto", "display": "block"}
                    )
                ),
                dbc.CardBody([
                    html.H6('Invites received'),
                    html.H2(id='content-msg-in', children="000")
                ], style={'textAlign':'center'})
            ]),
        ], width=2),
        dbc.Col([
            dbc.Card([
               # dbc.CardHeader(Lottie(options=options, width="53%", height="53%", url=url_msg_out)),
                  dbc.CardHeader(
                    html.Div(
                        Lottie(options=options, url=url_msg_out),
                        style={"width": "180px", "height": "150px", "margin": "0 auto", "display": "block"}
                    )
                ),
                dbc.CardBody([
                    html.H6('Invites sent'),
                    html.H2(id='content-msg-out', children="000")
                ], style={'textAlign': 'center'})
            ]),], width=2),
        dbc.Col([
            dbc.Card([
                #dbc.CardHeader(Lottie(options=options, width="25%", height="25%", url=url_reactions)),
                   dbc.CardHeader(
                    html.Div(
                        Lottie(options=options, url=url_reactions),
                        style={"width": "180px", "height": "150px", "margin": "0 auto", "display": "block"}
                    )
                ),
                dbc.CardBody([
                    html.H6('Reactions'),
                    html.H2(id='content-reactions', children="000")
                ], style={'textAlign': 'center'})
            ]),
        ], width=2),
    ],className='mb-2'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='line-chart', figure={}, config={'displayModeBar':False}),
                ])
            ]),
        ], width=5),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='bar-chart', figure={}, config={'displayModeBar':False}), 
                ])
            ]),
        ], width=5),
    ],className='mb-2'),
   dbc.Row([
    dbc.Col([
        dbc.Card([
            dbc.CardBody([
                dcc.Graph(id='pie-chart', figure={}),
            ])
        ]),
    ], width=6),

    dbc.Col([
        dbc.Card([
            dbc.CardBody([
                dcc.Graph(
                    id='wordcloud',
                    figure={},
                    config={'displayModeBar': False},
                    style={'marginBottom': '25px'} 
                ),
                html.P(
                    "The word cloud shows the most common job titles among participants. "
                    "Larger words appear more often in the dataset, while smaller words are less frequent. "
                    "This makes it easy to spot which positions are most dominant — for example, titles like "
                    "“Software Engineer” or “Data Analyst” may stand out as the most common roles.",
                    style={
                        'textAlign': 'center',
                        'fontStyle': 'italic',
                        'fontSize': '14px'
                        
                    }
                )
            ])
        ]),
    ], width=4),
], className='mb-2'),
], fluid=True)

'''
***********Used for other graphs ******************
dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='TBD', figure={}),
                ])
            ]),
        ], width=3)
'''
#updating the 5 number cards

@app.callback(
    Output('content-connections','children'),
    Output('content-companies','children'),
    Output('content-msg-in','children'),
    Output('content-msg-out','children'),
    Output('content-reactions','children'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)

def update_small_cards(start_date, end_date):
    # Connections
    dff_c = df_cnt.copy()

    dff_c = dff_c[(dff_c['Connected On']>=start_date) & (dff_c['Connected On']<=end_date)]
    conctns_num = len(dff_c)
    compns_num = len(dff_c['Company'].unique())

    # Invitations
    dff_i = df_invite.copy()
    dff_i = dff_i[(dff_i['Sent At']>=start_date) & (dff_i['Sent At']<=end_date)]
    # print(dff_i)
    in_num = len(dff_i[dff_i['Direction']=='INCOMING'])
    out_num = len(dff_i[dff_i['Direction']=='OUTGOING'])

    # Reactions
    dff_r = df_react.copy()
    dff_r = dff_r[(dff_r['Date']>=start_date) & (dff_r['Date']<=end_date)]
    reactns_num = len(dff_r)

    return conctns_num, compns_num, in_num, out_num, reactns_num


def update_small_cards(start_date, end_date):
    # Connections
    dff_c = df_cnt.copy()

    dff_c = dff_c[(dff_c['Connected On']>=start_date) & (dff_c['Connected On']<=end_date)]
    conctns_num = len(dff_c)
    compns_num = len(dff_c['Company'].unique())

    # Invitations
    dff_i = df_invite.copy()
    dff_i = dff_i[(dff_i['Sent At']>=start_date) & (dff_i['Sent At']<=end_date)]
    # print(dff_i)
    in_num = len(dff_i[dff_i['Direction']=='INCOMING'])
    out_num = len(dff_i[dff_i['Direction']=='OUTGOING'])

    # Reactions
    dff_r = df_react.copy()
    dff_r = dff_r[(dff_r['Date']>=start_date) & (dff_r['Date']<=end_date)]
    reactns_num = len(dff_r)

    return conctns_num, compns_num, in_num, out_num, reactns_num

# Line Chart ***********************************************************
@app.callback(
    Output('line-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)
def update_line(start_date, end_date):
    dff = df_cnt.copy()
    # Filter by date range
    dff = dff[(dff['Connected On'] >= start_date) & (dff['Connected On'] <= end_date)]
    # Count connections per month
    month_counts = dff['month'].value_counts().sort_index()
    # Prepare DataFrame for plotting
    plot_df = pd.DataFrame({
        'month': month_counts.index,
        'Total connections': month_counts.values
    })
    # Ensure months are ordered correctly (Jan, Feb, ...)
    plot_df['month'] = pd.Categorical(plot_df['month'], categories=list(calendar.month_abbr)[1:], ordered=True)
    plot_df = plot_df.sort_values('month')

    fig_line = px.line(
        plot_df,
        x='month',
        y='Total connections',
        template='ggplot2',
        title="Total Connections by Month Name"
    )
    fig_line.update_traces(mode="lines+markers", fill='tozeroy', line={'color':'blue'})
    fig_line.update_layout(margin=dict(l=20, r=20, t=30, b=20))

    return fig_line

# Bar Chart ************************************************************
@app.callback(
    Output('bar-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)
def update_bar(start_date, end_date):
    dff = df_cnt.copy()
    dff = dff[(dff['Connected On'] >= start_date) & (dff['Connected On'] <= end_date)]
    # Drop missing company names
    dff = dff.dropna(subset=['Company'])
    # taking only 6 markers
    company_counts = dff['Company'].value_counts().head(6).reset_index()
    company_counts.columns = ['Company', 'Total connections']

    fig_bar = px.bar(
        company_counts,
        x='Total connections',
        y='Company',
        template='ggplot2',
        orientation='h',
        title="Top 6 Companies by Connections"
    )
    fig_bar.update_yaxes(tickangle=45)
    fig_bar.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_bar.update_traces(marker_color='blue')

    return fig_bar

# Pie Chart ************************************************************
@app.callback(
    Output('pie-chart','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)
def update_pie(start_date, end_date):
    dff = df_msg.copy()
    dff = dff[(dff['DATE']>=start_date) & (dff['DATE']<=end_date)]
    msg_sent = len(dff[dff['FROM']=='Adam Schroeder'])
    msg_rcvd = len(dff[dff['FROM'] != 'Adam Schroeder'])
    fig_pie = px.pie(names=['Sent','Received'], values=[msg_sent, msg_rcvd],
                     template='ggplot2', title="Messages Sent & Received"
                     )
    fig_pie.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    fig_pie.update_traces(marker_colors=['red','blue'])

    return fig_pie

# Word Cloud ************************************************************
@app.callback(
    Output('wordcloud','figure'),
    Input('my-date-picker-start','date'),
    Input('my-date-picker-end','date'),
)
def update_wordcloud(start_date, end_date):
    # Filter data within date range
    dff = df_cnt[(df_cnt['Connected On'] >= start_date) & (df_cnt['Connected On'] <= end_date)]
    
    # Drop missing values and convert to string
    dff_positions = dff['Position'].dropna().astype(str)

    # If there’s no text data, return a blank figure
    if dff_positions.empty:
        return px.imshow(np.zeros((10,10)), title="No Position Data Found")

    # Generate the word cloud from positions
    my_wordcloud = WordCloud(
        background_color='white',
        width=800,
        height=400
    ).generate(' '.join(dff_positions))

    # Convert the WordCloud image to an array
    img_array = my_wordcloud.to_array()

    # Display the image with Plotly
    fig_wordcloud = px.imshow(img_array, title="Most Frequent Job Titles")
    fig_wordcloud.update_layout(
        margin=dict(l=20, r=20, t=40, b=20),
        xaxis_visible=False,
        yaxis_visible=False
    )

    return fig_wordcloud


webbrowser.open("http://127.0.0.1:8002")
app.run(mode='jupyterlab', port=8002)